# Extract per gene toy dataset

This generates per gene data-set and surveys how many SNPs in each generated dataset.

In [1]:
%revisions -s -n 10

Revision,Author,Date,Message
d4e8a8c,Gao Wang,2018-05-24,Update documentation


In [ ]:
[global]
parameter: n = 200
outdir = path('~/Documents/GTExV8/Toys')
tissue = 'Thyroid'

[extract]
genes = get_output(f'shuf {outdir:d}/all_genes.txt | head -{n}').strip().split('\n')
input: for_each = 'genes', concurrent = True
output: f"{outdir}/{tissue}.{path(_genes):b}.RDS"
R: expand = '${ }'

    load_data = function(genotype_file, expr_file, cov_file, geno_table, expr_table, cov_table) {
            library(rhdf5)
            geno = h5read(genotype_file, geno_table)
            gdata = data.frame(geno$block0_values)
            colnames(gdata) = geno$axis1
            rownames(gdata) = geno$axis0
            #
            expr = h5read(expr_file, expr_table)
            edata = data.frame(expr$block0_values)
            colnames(edata) = tools::file_path_sans_ext(expr$axis1)
            edata = data.frame(edata[, basename(geno_table)])
            rownames(edata) = expr$axis0
            colnames(edata) = basename(geno_table)
            #
            covariate <- h5read(cov_file, cov_table)
            cdata = data.frame(covariate$block0_values)
            colnames(cdata) = covariate$axis1
            rownames(cdata) = covariate$axis0
            cdata = t(cdata)
            # reorder
            edata = edata[match(rownames(cdata), rownames(edata)),]
            gdata = gdata[rownames(cdata),]
            return(list(X = as.matrix(gdata), y = as.vector(edata), Z = as.matrix(cdata), chrom = strsplit(colnames(gdata)[1], '_')[[1]][1], 
                        pos = as.integer(do.call(rbind, strsplit(colnames(gdata), '_'))[,2])))
        }
    genotype_file = "GTExV8.genotype.cis.h5"
    expr_file = "GTExV8.expression.h5"
    cov_file = "GTExV8.covariates.h5"
    ## 
    geno_table = "${_genes}"
    expr_table = "/${tissue}"
    cov_table = "/${tissue}"
    dat = load_data(genotype_file, expr_file, cov_file, geno_table, expr_table, cov_table)
    saveRDS(dat, ${_output:r})

To get an idea how many variants are there:

In [ ]:
[count_var]
input: glob.glob('~/Documents/GTExV8/Toys/*.RDS'), group_by = 1
output: '~/Documents/GTExV8/Toys/counts.txt'
R: expand = True, stdout = _output
  cat(paste0({_input:br}, ncol(readRDS({_input:r})$X),'\n'))

In [1]:
%cd ~/Documents/GTExV8/Toys

/home/gaow/Documents/GTExV8/Toys

In [2]:
dat = scan('counts.txt')
summary(dat)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   3022    6450    7226    7217    8033   11999 